# Simple dispatching optimization

In [ ]:
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from os import listdir
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
use_raw_price_data = False
now = datetime(2024,10,5,0,0,tzinfo=ZoneInfo('Australia/Melbourne'))
end = now + timedelta(hours=5*24) # 5 day optimization horizon

# Load in wind & price data
Wind data first

In [ ]:
dfw = pd.read_csv('data/sam_export_2024.csv',header=1)
dfw['utc'] = pd.to_datetime(dfw[['Year','Month','Day','Hour']]).dt.tz_localize('UTC') # NASA POWER | DAV provides UTC time
dfw['local'] = dfw['utc'].dt.tz_convert('Australia/Melbourne') # localize to UTC+10
dfw.set_index('local',inplace=True)

Price data next...

In [ ]:
if use_raw_price_data:
    csv_files = [f for f in listdir('data/') if f.endswith('.CSV') if f.startswith('PUBLIC_PRICES')]
    df_prices = []
    for f in csv_files:
        dfp = pd.read_csv(f'data/{f}',header=1,skipfooter=1,usecols=range(3,18))
        dfp.rename(columns={'2':"RUNNO_I_Think"},inplace=True) # I think that this is the real run number due to the duplicats
        most_recent_run = dfp['RUNNO_I_Think'].unique().max()

        # get most recent run and the correct region (VIC for this exercise)
        dfp = dfp[dfp['REGIONID'].str.contains('VIC') & 
                            (dfp['RUNNO_I_Think']==most_recent_run)]
        dfp['timestamp'] = pd.to_datetime(dfp.loc[:,'SETTLEMENTDATE']).dt.tz_localize('UTC') # set a datetime column
        dfp['timestamp'] = dfp['timestamp'].dt.tz_convert('Australia/Melbourne')
        dfp['RRP'] = dfp['RRP'].apply(lambda x: float(x))
        df_prices.append(dfp)

    df_prices = pd.concat(df_prices).sort_values(by='timestamp')
    df_prices.set_index('timestamp',inplace=True)
    df_prices.rename(columns={'RRP':'Settlement Price'},inplace=True)
    df_prices['Settlement Price'].to_csv('data/price_data.csv')
else:
    df_prices = pd.read_csv('data/price_data.csv',parse_dates=['timestamp'])
    df_prices.set_index('timestamp',inplace=True)

Upsample and adjust now/end to the bounds of the wind data

In [ ]:
dfw = dfw[(dfw.index<=end) & (dfw.index>=now)]
dfw_5min = dfw.resample("5min").interpolate("linear") # upsample wind to five minutes
now = max(dfw.index[0],now)
end = min(dfw.index[-1],end)
df_prices = df_prices[(df_prices.index<=end) & (df_prices.index>=now)] # ensure prices and wind have same domain

Plot

In [ ]:
fig, ax1 = plt.subplots()
# dfw.plot(ax=ax1,x='local',y='Wind speed at 100m',color='red')
ln1 = ax1.plot(dfw_5min.index,dfw_5min['Wind speed at 100m'],label='Wind speed (5 min)')
ax1.set_xlabel('Date/time')
ax1.set_ylabel('Wind speed (m/s)')

ax2 = ax1.twinx()
ln2 = ax2.plot(df_prices.index,df_prices['Settlement Price'],color='red',label='Price')
ax2.set_ylabel("Prices (AUD/MWh)", color="red")
ax2.tick_params(axis="y", labelcolor="red")

# Combine legends
lines = ln1 + ln2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc="best")

Export data to csv files

## Model setup
After SAM is run with the nine turbines, the maximum generation is available $P_{k}^{max}$. This data will be hourly and will need to be upsampled. 

In [ ]:
df_pmax = pd.read_csv('data/wind_farm_production.csv')
df_pmax['Time stamp'] = pd.to_datetime("2024 " + df_pmax['Time stamp'], format="%Y %b %d, %I:%M %p")
df_pmax['Time stamp'] = df_pmax['Time stamp'].dt.tz_localize('UTC')
df_pmax['Time stamp'] = df_pmax['Time stamp'].dt.tz_convert('Australia/Melbourne')
df_pmax.set_index('Time stamp',inplace=True)
df_pmax = df_pmax[(df_pmax.index<=end) & (df_pmax.index>=now)]
df_pmax_5min = df_pmax.resample("5T").interpolate("linear") # upsample wind to five minutes

fig, ax1 = plt.subplots()
# dfw.plot(ax=ax1,x='local',y='Wind speed at 100m',color='red')
ln1 = ax1.plot(df_pmax_5min.index,df_pmax_5min['System power generated | (kW)'],
               color='blue',label='Generated power (kW)')
ax1.set_xlabel('Date/time')
ax1.set_ylabel('Generated power (kW)',color='blue')
ax1.tick_params(axis="y", labelcolor="blue")

ax2 = ax1.twinx()
ln2 = ax2.plot(df_prices.index,df_prices['Settlement Price'],color='red',label='Price')
ax2.set_ylabel("Prices (AUD/MWh)", color="red")
ax2.tick_params(axis="y", labelcolor="red")

# Combine legends
lines = ln1 + ln2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc="best")

# df_pmax_5min.plot(y='System power generated | (kW)')

In [ ]:
s0,s_max = 100e3, 150e3 #kWh
Cs,Cg = 0,0 # no costs for now. Ignoring ramp cost in favor of a constraint
P_dis_max, P_chg_max,P_buy_max = 100e3,100e3,100e3 # kW 
Δt = 5.0/60.0 # hours
P_ramp_max = 0.2*P_dis_max/Δt # kW/hr
P_max = df_pmax_5min['System power generated | (kW)'].values
prices = df_prices['Settlement Price'].values/1e3 # $/kWh

N = df_prices.shape[0]
model = gp.Model("Wind with battery dispatching")
Ps = model.addVars(range(N),vtype=gp.GRB.CONTINUOUS,lb=0,ub=P_dis_max,name='Power sold')
Pg = model.addVars(range(N),vtype=gp.GRB.CONTINUOUS,lb=0,ub=P_chg_max,name='Power generated')
Pb = model.addVars(range(N),vtype=gp.GRB.CONTINUOUS,lb=0,ub=P_buy_max,name='Power generated')
S = model.addVars(range(N),vtype=gp.GRB.CONTINUOUS,lb=0,ub=s_max,name='Stored energy')

# setting the initials to zero since they are in the past
model.addConstr(S[0]==s0)
model.addConstr(Ps[0]==0)
model.addConstr(Pg[0]==0)
model.addConstr(Pb[0]==0)

# storage constraints
model.addConstrs((S[ii] == S[ii-1] - Ps[ii]*Δt + Pg[ii]*Δt + Pb[ii]*Δt for ii in range(1,N)))

# power capacity constraints
model.addConstrs((Pg[ii]<=P_max[ii] for ii in range(1,N)))
model.addConstrs((Pg[ii]+Pb[ii]<=P_chg_max for ii in range(1,N)))

# ramping constraint (linearized absolute value)
model.addConstrs(((Ps[ii]-Pb[ii]) - (Ps[ii-1] - Pb[ii-1]) <= P_ramp_max*Δt for ii in range(1,N))) 
model.addConstrs(((Ps[ii-1]-Pb[ii-1]) - (Ps[ii] - Pb[ii]) <= P_ramp_max*Δt for ii in range(1,N)))

# objective (revenue for now)
model.setObjective(Δt*gp.quicksum((  prices[ii]*(Ps[ii]-Pb[ii]) - Cg*Pg[ii] - Cs*Ps[ii] for ii in range(1,N))),sense=gp.GRB.MAXIMIZE)



In [ ]:
model.optimize()

In [ ]:
power_sold = []
power_generated = []
power_bought = []
storage = []
revenue = 0.0
for ii in range(N):
    power_sold += [Ps[ii].X]
    power_generated += [Pg[ii].X]
    power_bought += [Pb[ii].X]
    storage += [S[ii].X]
    revenue += prices[ii]*(Ps[ii].X-Pb[ii].X)*Δt

power_generated = np.array(power_generated)
power_bought = np.array(power_bought)
power_sold = np.array(power_sold)
storage = np.array(storage)
ramp_rate = np.diff(power_sold-power_bought,prepend=np.nan)/Δt

In [ ]:
fig,ax = plt.subplots(nrows=4,figsize=(7,8))
time = df_prices.index
ax[0].plot(time,(power_sold-power_bought)/1e3,label='Net export')
ax[0].plot(time,power_generated/1e3,label='Generated')
# ax.plot(time,P_max-power_generated/1e3,label='Curtailed')
ax[0].set_ylabel('Power (MW)')
ax[0].set_xticklabels([]) 
ax[0].legend(loc='best')

ax[1].plot(time,storage/1e3,label='Storage',color='green')
ax[1].axhline(s_max/1000,ls='--',color='red',label='Limits')
ax[1].axhline(0,ls='--',color='red')
ax[1].set_xticklabels([]) 
ax[1].set_ylabel("Stored energy (MWh)")
ax[1].legend(loc='best')

ax[2].plot(time,prices*1e3,label='Prices')
ax[2].set_xticklabels([]) 
ax[2].set_ylabel("Prices ($/MWh)")

ax[3].plot(time,ramp_rate/1000.0/60.0,label='Ramp rate')
ax[3].set_ylabel("Ramp rate (Net MW sold/min)")
ax[3].axhline(P_ramp_max/1000/60.0,ls='--',color='red',label='Limits')
ax[3].axhline(-P_ramp_max/1000/60.0,ls='--',color='red')
ax[3].legend(loc='best')

fig.tight_layout()
ax[0].set_title(f'Objective: {model.objVal:.3e} AUD, Revenue: {revenue:.3e} AUD')

